# MNIST Estimator

The original is too big to load in. Strange. This one will have a lot less comments

In [1]:
# Import Dependancies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import time
import numpy as np

In [2]:
# Import Data
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# configure tf logging
tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
# Declare variables
data_dir = '/tmp/MNIST_data'
model_dir_root = '/tmp/mnist_estimators'
model_dir = os.path.join(model_dir_root, "deep_" + str(int(time.time())))

# size of each batch of training examples to learn
batch_size = 50

num_steps = 15000

global DATA_SETS
print("downloading and reading data sets...")
DATA_SETS = input_data.read_data_sets(data_dir)

downloading and reading data sets...
Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
def generate_input_fn(dataset, batch_size=batch_size):
    def _input_fn():
        X = tf.constant(dataset.images)
        Y = tf.constant(dataset.labels, dtype=tf.int32)
        image_batch, label_batch = tf.train.shuffle_batch(
            [X,Y],
            batch_size=batch_size,
            capacity=8*batch_size,
            min_after_dequeue=4*batch_size,
            enqueue_many=True
        )
        return {'pixels': image_batch}, label_batch
    return _input_fn

In [6]:
def define_and_run_dnn_classifier(num_steps, logdir, lr=.1, batch_size=40):
    """Run a DNN classifier"""
    
    feature_columns = [tf.contrib.layers.real_valued_column("pixels", dimension=784)]
    
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=10,
        hidden_units=[128, 32],
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
    )
    
    classifier.train(
        input_fn=generate_input_fn(
            DATA_SETS.train,
            batch_size=batch_size
        ),
        steps=num_steps
    )
    
    print("Finished running the deep training")
    print("evaluating DNN classifier accuracy")
    
    accuracy_score = classifier.evaluate(
        input_fn=generate_input_fn(
            DATA_SETS.test,
            batch_size=batch_size
        ),
        steps=100
    )['accuracy']
    
    print("DNN classifier accuracy: {0:f}".format(accuracy_score))

In [7]:
print("Running DNN Classifier")
define_and_run_dnn_classifier(
    num_steps,
    model_dir
)

Running DNN Classifier
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_estimators/deep_1505612789', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_estimators/deep_1505612789/model.ckpt.
INFO:tensorflow:loss = 93.1617, step = 1
INFO:tensorflow:global_step/sec: 130.56
INFO:tensorflow:loss = 15.0384, step = 101 (0.572 sec)
INFO:tensorflow:global_step/sec: 163.335
INFO:tensorflow:loss = 21.3306, step = 201 (0.613 sec)
INFO:tensorflow:global_step/sec: 246.875
INFO:tensorflow:loss = 8.10742, step = 301 (0.405 sec)
INFO:tensorflow:global_step/sec: 242.731
INFO:tensorflow:loss = 17.3937, step = 401 (0.409 sec)
INFO:tensorflow:global_step/sec: 249.662
INFO:tensor

INFO:tensorflow:global_step/sec: 248.528
INFO:tensorflow:loss = 0.545688, step = 8001 (0.403 sec)
INFO:tensorflow:global_step/sec: 253.177
INFO:tensorflow:loss = 7.66352, step = 8101 (0.395 sec)
INFO:tensorflow:global_step/sec: 253.543
INFO:tensorflow:loss = 0.755263, step = 8201 (0.394 sec)
INFO:tensorflow:global_step/sec: 252.716
INFO:tensorflow:loss = 1.06112, step = 8301 (0.400 sec)
INFO:tensorflow:global_step/sec: 231.142
INFO:tensorflow:loss = 4.06428, step = 8401 (0.428 sec)
INFO:tensorflow:global_step/sec: 196.464
INFO:tensorflow:loss = 4.52833, step = 8501 (0.509 sec)
INFO:tensorflow:global_step/sec: 261.763
INFO:tensorflow:loss = 3.18019, step = 8601 (0.382 sec)
INFO:tensorflow:global_step/sec: 261.517
INFO:tensorflow:loss = 1.07489, step = 8701 (0.383 sec)
INFO:tensorflow:global_step/sec: 260.509
INFO:tensorflow:loss = 4.36505, step = 8801 (0.384 sec)
INFO:tensorflow:global_step/sec: 243.815
INFO:tensorflow:loss = 0.755871, step = 8901 (0.414 sec)
INFO:tensorflow:global_step

INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [

In [8]:
# tensorboard --logdir=/tmp/mnist_estimators